In [ ]:
!pip install torch torchvision numpy matplotlib scikit-learn tqdm

import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {device}")

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

zip_train_path = '/content/drive/MyDrive/dataset/train.zip' 
zip_val_path   = '/content/drive/MyDrive/dataset/val.zip' 

base_dir  = '/content/temp_dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir   = os.path.join(base_dir, 'val')

def extract_zip(zip_path, dest_path):
    if not os.path.exists(dest_path):
        print(f"Membuat folder {dest_path}...")
        os.makedirs(dest_path, exist_ok=True)

        print(f"Mengekstrak {zip_path} ke {dest_path}...")
        !unzip -q "$zip_path" -d "$dest_path"
        print(f"Selesai mengekstrak ke {dest_path}")
    else:
        print(f"Folder {dest_path} lewati ekstrak")

if os.path.exists(zip_train_path):
    extract_zip(zip_train_path, train_dir)
else:
    print(f"File {zip_train_path} tidak ditemukan di Drive")

if os.path.exists(zip_val_path):
    extract_zip(zip_val_path, val_dir)
else:
    print(f"File {zip_val_path} tidak ditemukan di Drive")

DATA_DIR = base_dir
TRAIN_DIR = train_dir
VAL_DIR = val_dir

if os.path.exists(TRAIN_DIR):
    print(f"Isi {TRAIN_DIR}: {os.listdir(TRAIN_DIR)}")

    subfolders = os.listdir(TRAIN_DIR)
    if len(subfolders) > 0:
        first_sub = os.path.join(TRAIN_DIR, subfolders[0])
        if os.path.isdir(first_sub):
             print(f"Isi dalam '{subfolders[0]}': {os.listdir(first_sub)[:5]}")

if os.path.exists(VAL_DIR):
    print(f"Isi {VAL_DIR}: {os.listdir(VAL_DIR)}")

In [6]:
class FFTTransform:
    def __call__(self, img):
        #Grayscale
        img_gray = img.convert('L')
        img_array = np.array(img_gray)

        #Fast Fourier Transform
        f = np.fft.fft2(img_array)
        fshift = np.fft.fftshift(f) #frekuensi rendah ke tengah

        #Hitung Magnitude Spectrum
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)

        #Normalisasi ke range 0-255
        magnitude_spectrum = np.nan_to_num(magnitude_spectrum)
        ms_min = np.min(magnitude_spectrum)
        ms_max = np.max(magnitude_spectrum)

        # Scaling min-max
        if ms_max - ms_min > 0:
            img_fft = 255 * (magnitude_spectrum - ms_min) / (ms_max - ms_min)
        else:
            img_fft = np.zeros_like(magnitude_spectrum)

        img_fft = img_fft.astype(np.uint8)

        #Konversi PIL Image
        img_fft_pil = Image.fromarray(img_fft).convert("RGB")

        return img_fft_pil

# def visualize_fft_sample(image_path):
#     img = Image.open(image_path)
#     transformer = FFTTransform()
#     fft_img = transformer(img)

#     fig, ax = plt.subplots(1, 2, figsize=(10, 5))
#     ax[0].imshow(img)
#     ax[0].set_title("Original Image")
#     ax[1].imshow(fft_img)
#     ax[1].set_title("FFT Spectrum Input")
#     plt.show()

# visualize_fft_sample('/content/drive/MyDrive/dataset/train/train/ai/imgAI602.jpg')

In [ ]:
#MACOS
import os
import shutil

def remove_macos_artifacts(path):
    print(f"Membersihkan MacOS di: {path}")
    count_files = 0
    count_folders = 0

    for root, dirs, files in os.walk(path):
        if '__MACOSX' in dirs:
            rm_path = os.path.join(root, '__MACOSX')
            try:
                shutil.rmtree(rm_path)
                count_folders += 1
            except Exception as e:
                print(f"Gagal hapus {rm_path}: {e}")
            dirs.remove('__MACOSX')

        for file in files:
            if file.startswith('._'):
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    count_files += 1
                except Exception as e:
                    print(f"Gagal hapus {file_path}: {e}")

    print(f"Selesa hapus: {count_folders} folder __MACOSX{count_files}")

remove_macos_artifacts(DATA_DIR)

In [ ]:
image_datasets = {
    'train': torchvision.datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'val': torchvision.datasets.ImageFolder(VAL_DIR, data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print("DataLoader berhasil diperbarui tanpa file sampah MacOS.")
print(f"Total Train: {dataset_sizes['train']}")

In [ ]:
image_datasets = {
    'train': torchvision.datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'val': torchvision.datasets.ImageFolder(VAL_DIR, data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print("DataLoader berhasil diperbarui tanpa file sampah MacOS.")
print(f"Total Train: {dataset_sizes['train']}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

num_ftrs = model.classifier[1].in_features

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(num_ftrs, 2)
)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
import os
from PIL import Image

def validate_and_clean_images(root_dir):
    print(f"Memulai pengecekan integritas gambar di: {root_dir}")
    print("Proses ini mungkin memakan waktu 1-2 menit...")

    bad_files = 0
    checked_files = 0

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
                file_path = os.path.join(root, file)
                checked_files += 1

                try:
                    with Image.open(file_path) as img:
                        img.verify()
                except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
                    print(f"File rusak ditemukan: {file_path}")
                    try:
                        os.remove(file_path)
                        print(f"Berhasil dihapus")
                        bad_files += 1
                    except Exception as del_err:
                        print(f"Gagal menghapus: {del_err}")

    print("-" * 30)
    print(f"Total diperiksa: {checked_files}")
    print(f"Total file rusak dihapus: {bad_files}")

validate_and_clean_images(DATA_DIR)

In [ ]:
image_datasets = {
    'train': torchvision.datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'val': torchvision.datasets.ImageFolder(VAL_DIR, data_transforms['val'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=2),
    'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(f"Dataset di-refresh. Total Train: {dataset_sizes['train']}, Val: {dataset_sizes['val']}")

In [ ]:
model_trained, history = train_model(model, criterion, optimizer, num_epochs=10)

In [ ]:
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history['train_acc'], label='Train Accuracy', marker='o')
plt.plot(history['val_acc'], label='Validation Accuracy', marker='o')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history['train_loss'], label='Train Loss', marker='o')
plt.plot(history['val_loss'], label='Validation Loss', marker='o')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.show()

torch.save(model.state_dict(), 'final_model_efficientnet_fft.pth')
print("Model berhasil disimpan.")

from google.colab import files
print("Mempersiapkan download model terbaik...")

if os.path.exists('best_model_efficientnet_fft.pth'):
    files.download('best_model_efficientnet_fft.pth')
else:
    print("File best_model tidak ditemukan, download model final saja.")
    files.download('final_model_efficientnet_fft.pth')